In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import griddata
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.image as mpimg

# -----------------------------
# USER‑ADJUSTABLE SETTINGS
# -----------------------------
FIG_WIDTH = 10
FIG_HEIGHT = 8
# -----------------------------

# Load CSV
df = pd.read_csv("your_data.csv")

x = df.iloc[:, 0].values
y = df.iloc[:, 1].values
z = df.iloc[:, 2].values

# --- User Input for Plot Bounds ---
def get_bound(prompt_text, default_value):
    user_input = input(f"{prompt_text} (press Enter for {default_value}): ")
    return float(user_input) if user_input.strip() else default_value

xmin = get_bound("Enter xmin", x.min())
xmax = get_bound("Enter xmax", x.max())
ymin = get_bound("Enter ymin", y.min())
ymax = get_bound("Enter ymax", y.max())

# --- Optional background image with auto-scaling ---
use_bg = input("Use a background image? (y/n): ").strip().lower() == "y"

if use_bg:
    img_path = input("Enter image filename: ").strip()
    bg = mpimg.imread(img_path)
    h, w = bg.shape[:2]

    print("\nProvide TWO matching points:")
    print("Point A (real-world coords):")
    x1 = float(input("  x1: "))
    y1 = float(input("  y1: "))

    print("Point A (pixel coords):")
    px1 = float(input("  px1: "))
    py1 = float(input("  py1: "))

    print("\nPoint B (real-world coords):")
    x2 = float(input("  x2: "))
    y2 = float(input("  y2: "))

    print("Point B (pixel coords):")
    px2 = float(input("  px2: "))
    py2 = float(input("  py2: "))

    # Compute scale factors
    scale_x = (x2 - x1) / (px2 - px1)
    scale_y = (y2 - y1) / (py2 - py1)

    # Compute offset
    offset_x = x1 - px1 * scale_x
    offset_y = y1 - py1 * scale_y

    # Compute full image extent
    img_xmin = offset_x
    img_xmax = offset_x + w * scale_x
    img_ymin = offset_y
    img_ymax = offset_y + h * scale_y

# --- Ask user if they want point labels ---
label_choice = input("Label each data point with its intensity value? (y/n): ").strip().lower()
label_points = (label_choice == "y")

# Create grid
grid_x, grid_y = np.mgrid[
    xmin:xmax:200j,
    ymin:ymax:200j
]

grid_z = griddata((x, y), z, (grid_x, grid_y), method='cubic')

colors = ["blue", "purple", "red"]
cmap = LinearSegmentedColormap.from_list("blue_red_only", colors)

plt.figure(figsize=(FIG_WIDTH, FIG_HEIGHT))

# Draw background image
if use_bg:
    plt.imshow(
        bg,
        extent=(img_xmin, img_xmax, img_ymin, img_ymax),
        origin='lower',
        alpha=1.0
    )

# Draw heatmap
plt.imshow(
    grid_z.T,
    extent=(xmin, xmax, ymin, ymax),
    origin='lower',
    cmap=cmap,
    alpha=0.75
)

cbar = plt.colorbar()
cbar.set_label("Light Intensity (Lux)", fontsize=12)

plt.xlabel("X Position (ft)")
plt.ylabel("Y Position (ft)")
plt.title("Light Intensity Heatmap (Lux)")

if label_points:
    for xi, yi, zi in zip(x, y, z):
        plt.text(xi, yi, f"{zi:.1f}", color="black", fontsize=7,
                 ha="center", va="center")

plt.tight_layout()
plt.show()